## Load Data

In [2]:
import numpy as np
from os.path import dirname, join as pjoin
import scipy.io as sio
import pandas as pd
import tensorflow as tf

DATA_PATH = './data'
DATA_FILE_NAME = 'Brugge_en_d.mat'

data_dir = pjoin(DATA_PATH, DATA_FILE_NAME)
mat_contents = sio.loadmat(data_dir)

data = mat_contents['en_d'][0, 0]

tf.debugging.set_log_device_placement(True)

In [ ]:
# TIME
# series of days (498 days between day 0-3648), not continuous
# not only integer, contains several float
print("TIME")
print("numpy array shape: {}".format(data['TIME'].shape))

In [ ]:
# WBHP
# Well Bottom Hole Pressure
# daily data 498 days for 104 models
print("WBHP")
print("numpy array shape: {}".format(data['WBHP'].shape))
# Producers P1-P20
# and Injectors I1-I10
# shape of each is 498x104 double
print("Producer P1 numpy array shape: {}".format(data['WBHP'][0,0]['P1'].shape))
print("Injectre I1 numpy array shape: {}".format(data['WBHP'][0,0]['I1'].shape))

In [ ]:
# WWCT
# Well Water Cut: water/entire_liquid
print("WWCT")
print("numpy array shape: {}".format(data['WWCT'].shape))
# Producers P1-P20
print("Producer P1 numpy array shape: {}".format(data['WWCT'][0,0]['P1'].shape))

In [ ]:
# WOPR
# Well Oil Production Rate: bbl/day
# What we want to forecast, this will be used as label
print("WOPR")
print("numpy array shape: {}".format(data['WOPR'].shape))
# Producers P1-P20, index 0-19
print("Producer P1 numpy array shape: {}".format(data['WOPR'][0,0]['P1'].shape))

In [ ]:
# WWPR
# Well Water Production Rate
print("WWPR")
print("numpy array shape: {}".format(data['WWPR'].shape))
# Producers P1-P20
print("Producer P1 numpy array shape: {}".format(data['WWPR'][0,0]['P1'].shape))

In [ ]:
# WWIR
# Well Water Injection Rate
print("WWIR")
print("numpy array shape: {}".format(data['WWIR'].shape))
# Injectors I1-I10
print("Injector 1 numpy array shape: {}".format(data['WWIR'][0,0]['I1'].shape))

In [ ]:
# FOPT
# Field Oil Production Total: sum of WOPR * days
print("FOPT")
print("numpy array shape: {}".format(data['FOPT'].shape))
# FWPT
# Field Water Production Total
print("FWPT")
print("numpy array shape: {}".format(data['FWPT'].shape))
# FPR
# Field Production Rate: FOPR + FWPR
print("FPR")
print("numpy array shape: {}".format(data['FPR'].shape))

In [ ]:
# WWBT
# Well W B Total
print("WWBT")
print("numpy array shape: {}".format(data['WWBT'].shape))
# Producers P5,P11-P20
print("Producer P5 numpy array shape: {}".format(data['WWBT'][0,0]['P5'].shape))

# Data Preparation

각 model 별로 아래와 같은 Dataframe 을 만들자.

|date|WOPR|WBHP|WWCT|WWPR|
|--|--|--|--|--|
|0.0|0.0|0.0|0.0|0.0|
|1.0|0.0|0.0|0.0|0.0
| . | . | . | . | . |
|3641.0|1998.772|1393.615|0.001552|3.107502|
|3648.0|1998.775|1393.951|0.001551|3.105065|

In [4]:
#  'well_num' => dfs_dic
dic_wells = {}
contain_null = False
for well_index in range(20): # well, Producer P1-P20
    # 'model_num' => dataframe
    dic_models = {}
    well_key = 'P' + str(well_index+1)
    for model_index in range(104): # model, model 1-104
        well_data = np.array([
            data['WOPR'][0,0][well_key][:,model_index],
            data['WBHP'][0,0][well_key][:,model_index],
            data['WWCT'][0,0][well_key][:,model_index],
            data['WWPR'][0,0][well_key][:,model_index]
          ])
        # col1: WOPR, col2: WBHP, col3: WWCT, col4: WWPR
        # row1: day1, ... row 498: day3648
        well_data = well_data.T
        df = pd.DataFrame(data=well_data,
                          index=data['TIME'].flatten(),
                          columns=['WOPR', 'WBHP', 'WWCT', 'WWPR'])
        df.index.name = 'date'
        if df.isnull().values.any():
            contain_null = True
        dic_models[str(model_index+1)] = df
        
    dic_wells[str(well_index+1)] = dic_models
    
contain_null, dic_wells['20'].keys()

(False,
 dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104']))

In [ ]:
# [well_num][model_num]
print(dic_wells['1']['1'].head())
print(dic_wells['1']['1'].tail())
print(dic_wells['20']['1'].head())
print(dic_wells['20']['1'].tail())

In [ ]:
# plot well 20, model 1 data
import matplotlib.pyplot as plt

ax = plt.gca()

for i in range(104):
    df = dic_wells['20'][str(i + 1)]
    # model #104 is true model
    # while model #1-103 is equivalent model
    if i == 103:
        df.plot(y='WOPR', ax=ax, color='red', legend=False)
    else:
        df.plot(y='WOPR', ax=ax, color='gray', legend=False)

plt.show()

# Data Preprocessing for LSTM

LSTM에 맞게 data를 전처리하자.
Multivariate, multi-step Forecasting을 하는 것이 목적이다. 

Input dimension은 4로,

- WOPR
- WBHP
- WWCT
- WWPR

을 사용한다.

sequence(step size)는 5로 한다. 즉, 이전 5개(t-1, t-2, t-3, t-4, t-5)의 time step의 data를 input으로 하여 현재 time step t의 WOPR을 예측한다.

In [ ]:
def series_to_supervised(dataframe, n_in=1, n_out=1, dropnan=True):
    '''
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a dataframe
        n_in: Number of lag observation as input, number of sequence
        n_out: Number of observations as output, output dimension
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    '''
    n_vars = len(dataframe.columns)
    cols, names = list(), list()
    # input sequence (t-n, ..., t-1)
    for i in range(n_in, 0, -1):
        cols.append(dataframe.shift(i))
        names += [('var%d(t-%d)' %(j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, ... , t+n)
    for i in range(0, n_out):
        cols.append(dataframe.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1,i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
target_df = dic_wells['20']['103']

reframed = series_to_supervised(target_df, n_in=5, n_out=1)
print(reframed.loc[:, :].shape)
print(reframed.tail())

In [ ]:
eff_time_step = len(reframed[['var1(t)']])
train_size = int(0.3 * eff_time_step)
test_size = eff_time_step - train_size

train_split = data['TIME'][train_size+4][0]
# print(reframed.loc[:train_split,:].shape)
scaler = scaler.fit(reframed.loc[:train_split,:])

reframed.loc[:, :] = scaler.transform(reframed.loc[:, :])
print(reframed.tail())

training set과 test set을 분리하자.

In [ ]:
data_to_numpy = reframed.to_numpy()
train, test = data_to_numpy[:train_size, :], data_to_numpy[train_size:, :]

train_X, train_y = train[:, :-4], train[:, -4]
test_X, test_y = test[:, :-4], test[:, -4]
# reshape input to be 3D [samples, timesteps, features]
input_timesteps = 5
input_dimension = 4
train_X = train_X.reshape((train_X.shape[0], input_timesteps, input_dimension))
test_X = test_X.reshape((test_X.shape[0], input_timesteps, input_dimension))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# Define and Fit the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

# design the network
model = Sequential()
model.add(LSTM(25, input_shape=(input_timesteps, input_dimension)))
model.add(Dense(1))

model.summary()

In [ ]:
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=24, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

# Evaluate the model

In [ ]:
# make a prediction
y_hat = model.predict(test_X)
test_X = test_X.reshape(test_X.shape[0], input_timesteps * input_dimension)

scale = 1/scaler.scale_[-4]

In [ ]:
evaluate_array = np.array([
    np.concatenate((train_y, y_hat), axis=None),
    np.concatenate((train_y, test_y), axis=None)
])

df = pd.DataFrame(data=evaluate_array.T, columns=['prediction', 'true'])

plt.plot(data['TIME'][5:], df[['prediction']] * scale, linestyle='-', label='prediction')
plt.plot(data['TIME'][5:], df[['true']] * scale, linestyle='-',label = 'true')
plt.legend()
plt.show()

In [ ]:
# invert scaling for forecast
inv_y_hat = np.concatenate((y_hat, test[:, 1:]), axis = 1)
print(y_hat.shape)
# print(y_hat)
# print(test_X.shape)
# print(test_X)
print(inv_y_hat.shape)
inv_y_hat = scaler.inverse_transform(inv_y_hat)
inv_y_hat = inv_y_hat[:, 0]


# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
# print(test_y.shape, test_X[:, 1:].shape)
inv_y = np.concatenate((test_y, test[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, 0]

In [ ]:
# calculate RMSE
from math import sqrt
from sklearn.metrics import mean_squared_error

rmse = sqrt(mean_squared_error(inv_y, inv_y_hat))
print('Test RMSE: %.3f' % rmse)